In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fasttextmodel/fasttextvec.pt
/kaggle/input/fasttextmodel/explainations10.pt
/kaggle/input/fasttextmodel/IMDB_split.pt
/kaggle/input/fasttextmodel/explainations50.pt
/kaggle/input/fasttextmodel/model_epoch0.pt
/kaggle/input/fasttextmodel/FastText.model.wv.vectors_ngrams.npy
/kaggle/input/fasttextmodel/FastText.model
/kaggle/input/fasttextmodel/IMDB_cleaned.pt
/kaggle/input/fasttextmodel/data.csv
/kaggle/input/fasttextmodel/explainations.pt


In [1]:
import torch
from tqdm import trange

In [2]:
part1 = torch.load('../input/fasttextmodel/explainations.pt')
part2 = torch.load('../input/fasttextmodel/explainations10.pt')
part3 = torch.load('../input/fasttextmodel/explainations50.pt')

In [3]:
all_exps = part1 + part2 + part3
print(len(all_exps))

100


In [6]:
for i in trange(len(all_exps)):
    if i ==0:
        vocab = all_exps[i]['evals'].T[0]
    else:
        vocab = np.concatenate((vocab, all_exps[i]['evals'].T[0]))
        if i%100==0:
            vocab = np.unique(vocab)

vocab_count = {word:[0,0,0,0,0,0] for word in vocab}
for i in trange(len(all_exps)):
    evals = all_exps[i]['evals']
    for word in evals:
        vocab_count[word[0]][all_exps[i]['label']] += 1
        vocab_count[word[0]][all_exps[i]['label']+3] += float(word[1])
vocab_details = pd.DataFrame(vocab_count)
vocab = vocab_details.columns
vocab_details = vocab_details.values
vocab_details = np.concatenate((np.array([vocab]), vocab_details), axis=0).T

100%|██████████| 100/100 [00:00<00:00, 2612.39it/s]


In [7]:
## POSITIVES - label 1 but then, here it is 1+1+3=5

n = 5
proba = np.sum(vocab_details.T[1:], axis=0)
indexes = np.argsort(vocab_details.T[n]*(vocab_details.T[n-3]/proba))[::-1]
print(vocab_details.T[0][indexes][:100])

['mario' 'young' 's' 'war' 'man' 'police' 'coburn' 'kane' 'film' 'father'
 'director' 'life' 'times' 'wonderful' 'mother' 'one' 'world' 'jimmy'
 'well' 'us' 'story' 'comedy' 'castle' 'hard' 'may' 'different' 'cold'
 'musical' 'oz' 'doug' 'upon' 'men' 'first' 'years' 'mumbai' 'glory'
 'torture' 'mountain' 'sir' 'great' 'dresser' 'ryan' 'little' 'fulci'
 'anand' 'mike' 'characters' 'hitchcock' 'bowser' 'home' 'king' 'michael'
 'cristy' 'makes' 'german' 'period' 'either' 'thrillers' 'killer'
 'beginning' 'satya' 'gun' 'hawkings' 'elements' 'take' 'sturgis' 'civil'
 'gives' 'directing' 'espionage' 'peace' 'count' 'farce' 'tone' 'amazing'
 'cinema' 'spoilers' 'mattei' 'theater' 'die' 'durbin' 'finds' 'city'
 'peach' 'reviewers' 'knowing' 'sharp' 'without' 'human' 'mouth' 'serial'
 'particularly' 'govind' 'unique' 'put' 'however' 'business' 'mukhsin'
 'friendship' 'soldiers']


In [8]:
## NEGATIVES - label 0 but then, here it is 0+1+3=4

n = 4
proba = np.sum(vocab_details.T[1:], axis=0)
indexes = np.argsort(vocab_details.T[n]*(vocab_details.T[n-3]/proba))[::-1]
print(vocab_details.T[0][indexes][:100])

['two' 'gem' 'alexandra' 'viewing' 'mormon' 'effects' 'crocky' 'crocodile'
 'tobe' 'lite' 'audience' 'cry' 'faust' 'husband' 'woman' 'midler' 'silly'
 'hooper' 'scripts' 'independence' 'convincing' 'maybe' 'beauty'
 'missionaries' 'boll' 'dutcher' 'surf' 'warehouse' 'eriksen' 'scene'
 'steele' 'loved' 'sense' 'nicholas' '2000' 'stella' 'ophelia' 'trolllike'
 'mare' 'came' 'blockbuster' 'hills' 'predictably' 'follow' 'jeep' '50'
 'carver' 'added' 'steal' 'care' 'release' 'chum' 'face' 'things' 'kid'
 'randall' 'novel' 'french' 'ripoffs' 'worn' 'fodder' 'halfpints'
 'accidentally' 'straight' 'enough' 'busty' 'initially' 'bonbons'
 'shocking' 'replacement' 'grainy' 'hoek' 'loss' 'portraying' 'computer'
 'im' 'bought' 'nothing' 'gary' 'degenerates' 'dignity' 'eating' 'sticker'
 'raise' 'necrophilia' 'collins' 'accent' 'interest' 'starts' 'flair'
 'stacie' 'together' 'malevolent' 'matured' 'floor' 'accuses'
 'allusionshomages' 'alvarado' 'improved' 'summon']
